## Prepare imports

In [1]:
import torch
import numpy as np 
from datasets.topological import DataModule, DataModuleConfig
import matplotlib.pyplot as plt
import pyvista as pv
from torch_geometric.data import Batch
import pyvista as pv

from models.encoder import BaseModel as Encoder
from kaolin.metrics.pointcloud import chamfer_distance

from load_configs import load_config

import matplotlib.pyplot as plt

DEVICE = "cuda:0"

from types import SimpleNamespace
from datasets import load_datamodule
import yaml
import json

from models.encoder import BaseModel as Encoder


encoder_config = load_config("./configs/config_encoder_topological.yaml")
dm = load_datamodule(encoder_config.data)


encoder = Encoder.load_from_checkpoint(f"./trained_models/{encoder_config.modelconfig.save_name}").to(DEVICE)




Processing...
Done!
Processing...
Done!
Processing...
Done!
Processing...
Done!
C:\Users\ernst\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\lightning\pytorch\utilities\migration\utils.py:56: The loaded checkpoint was produced with Lightning v2.3.3, which is newer than your current Lightning version: v2.2.3


In [2]:
encoder.layer.v.shape

torch.Size([3, 96])

## Load Models and Data

In [3]:
# batch_len = len(dm.test_dataloader())
test_ds = dm.test_ds

# For each class, grab 16 samples.
mfld_classes = test_ds.y.unique() 

test_data_list = []

for idx in mfld_classes: 
    test_data_list.extend([test_ds[test_ds.y == idx][i] for i in range(1)])


test_batch = Batch.from_data_list(test_data_list).to(DEVICE)
points_batch = test_batch.x.cpu().detach().view(-1,1024,3).numpy()


In [4]:
int_to_name = {
    0: "Sphere",
    1: "Torus",
    2: "Cube",
    3: "Mobius Strip"
}

In [5]:
losses = []
with torch.no_grad():
    for batch in dm.test_dataloader():
        batch = batch.cuda()
        batch_len = len(batch)
        # Compute ECT
        ect = encoder.layer(batch, batch.batch)

        # Reconstruct the batch
        x = encoder(ect).view(-1, encoder.modelconfig.num_dims)

        loss_cd = chamfer_distance(
            x.view(batch_len, -1, 3),
            batch.x.view(-1, 1024,3),
        )
        print(loss_cd)
        losses.extend([{"loss":l.item(), "Model":"ECT-MLP", "y":int_to_name[y.item()]} for l,y in zip(loss_cd,batch.y)])

tensor([0.0061, 0.0060, 0.0060, 0.0062, 0.0061, 0.0059, 0.0060, 0.0061, 0.0056,
        0.0063, 0.0058, 0.0058, 0.0058, 0.0062, 0.0062, 0.0058, 0.0061, 0.0067,
        0.0060, 0.0060, 0.0060, 0.0059, 0.0060, 0.0063, 0.0061, 0.0061, 0.0060,
        0.0060, 0.0055, 0.0062, 0.0060, 0.0067, 0.0058, 0.0066, 0.0065, 0.0058,
        0.0062, 0.0060, 0.0062, 0.0060, 0.0058, 0.0061, 0.0060, 0.0063, 0.0057,
        0.0060, 0.0060, 0.0063, 0.0061, 0.0059, 0.0061, 0.0081, 0.0060, 0.0058,
        0.0061, 0.0059, 0.0066, 0.0061, 0.0060, 0.0061, 0.0058, 0.0060, 0.0061,
        0.0061], device='cuda:0')
tensor([0.0060, 0.0061, 0.0058, 0.0065, 0.0058, 0.0060, 0.0061, 0.0063, 0.0056,
        0.0061, 0.0059, 0.0057, 0.0059, 0.0061, 0.0064, 0.0061, 0.0062, 0.0066,
        0.0062, 0.0059, 0.0059, 0.0060, 0.0059, 0.0060, 0.0059, 0.0058, 0.0064,
        0.0058, 0.0057, 0.0058, 0.0058, 0.0058, 0.0059, 0.0063, 0.0061, 0.0058,
        0.0060, 0.0059, 0.0067, 0.0060, 0.0065, 0.0068, 0.0066, 0.0064, 0.0068,
      

KeyboardInterrupt: 

In [ ]:
dm.test_ds

TopologicalDataset(12000)

# Add random rotation.

In [ ]:
from datasets.topological import RandomRotate
import torchvision.transforms as transforms


randrot = transforms.Compose(
            [
                RandomRotate(degrees=90, axis=0),
                RandomRotate(degrees=90, axis=1),
                RandomRotate(degrees=90, axis=2),
            ]
        )

In [ ]:
with torch.no_grad():
    for batch in dm.test_dataloader():
        batch = batch.cuda()
        batch_len = len(batch)
        # Compute ECT

        # R the batch
        rotbatch = Batch.from_data_list([randrot(batch[i]) for i in range(len(batch))])

        loss_cd = chamfer_distance(
            rotbatch.x.view(batch_len, -1, 3),
            batch.x.view(-1, 1024,3),
        )
        losses.extend([{"loss":l.item(), "Model":"RandomRotation", "y":int_to_name[y.item()]} for l,y in zip(loss_cd,batch.y)])

# Add random rotation.

In [ ]:
with torch.no_grad():
    for batch in dm.test_dataloader():
        batch = batch.cuda()
        batch_len = len(batch)
        # Compute ECT

        # R the batch
        rotbatch = Batch.from_data_list([randrot(batch[i]) for i in range(len(batch))])

        loss_cd = chamfer_distance(
            rotbatch.x.view(batch_len, -1, 3),
            batch.x.view(-1, 1024,3),
        )
        losses.extend([{"loss":l.item(), "Model":"RandomRotation", "y":int_to_name[y.item()]} for l,y in zip(loss_cd,batch.y)])

# Ground Truth

In [ ]:
# from datasets.topological import TopologicalDataset, DataModuleConfig
# from datasets.transforms import CenterTransform

# config1 = DataModuleConfig(root="./data/gt1",n_manifolds = 3000)
# config2 = DataModuleConfig(root="./data/gt2",n_manifolds = 3000)

# ds = TopologicalDataset(config1, split="test", pre_transform=CenterTransform())
# ds2 = TopologicalDataset(config2, split="test",pre_transform=CenterTransform())

# for x1,x2 in zip(ds,ds2):
#     loss_cd = chamfer_distance(
#             x1.x.view(-1, 1024, 3).cuda(),
#             x2.x.view(-1, 1024,3).cuda(),
#         )
#     assert x1.y.item() == x2.y.item()
#     print(loss_cd)
#     losses.extend([{"loss":loss_cd.item(), "Model":"GroundTruth", "y":int_to_name[x1.y.item()]}])


In [ ]:
import pandas as pd 
import seaborn as sns 

loss_df = pd.DataFrame(losses) 
loss_df["loss"] = loss_df["loss"] * 1e4

loss_df.head()


,loss,Model,y
0,78.502847,ECT-MLP,Sphere
1,63.796462,ECT-MLP,Sphere
2,60.910080,ECT-MLP,Sphere
3,63.405070,ECT-MLP,Sphere
4,59.450660,ECT-MLP,Sphere


In [ ]:
loss_df.groupby(["y","Model"]).agg(["mean","std"]).unstack().swaplevel(1,2,axis=1).sort_index(axis=1)

loss                                       
Model           ECT-MLP            RandomRotation             
                   mean        std           mean          std
y                                                             
Cube          75.979438  25.262602     312.152510    82.952938
Mobius Strip  40.317553  24.023995    4036.741922  2140.791053
Sphere        61.294812   4.396492      80.637017     7.528458
Torus         56.656536   9.848542    1525.561388   676.067514

In [ ]:
pl = pv.Plotter(shape=(1,2), window_size=[1600, 400],border=False,polygon_smoothing=True)

idx = 10
x1 = ds[idx]
x2 = ds2[idx]


pl.subplot(0, 0)
actor = pl.add_points(
    np.vstack([
        x2.x.reshape(-1, 3).cpu().detach().numpy(),
        x1.x.reshape(-1, 3).numpy(),
    ]),
    style="points",
    emissive=False,
    show_scalar_bar=False,
    render_points_as_spheres=True,
    # scalars=x1.x.reshape(-1, 3)[:, 2],
    point_size=5,
    ambient=0.2, 
    diffuse=0.8, 
    specular=0.8,
    specular_power=40, 
    smooth_shading=True
)

pl.background_color = "w"
pl.link_views()
pl.camera_position = "yz"
pos = pl.camera.position
pl.camera.position = (pos[0],pos[1],pos[2]+3)
pl.camera.azimuth = -45
pl.camera.elevation = 10

# create a top down light
light = pv.Light(position=(0, 0, 3), positional=True,
                cone_angle=50, exponent=20, intensity=.2)
pl.add_light(light)
pl.camera.zoom(1.3)
pl.screenshot("./figures/img/topological/reconstructed_pointcloud.png",transparent_background=True,scale=2)
pl.show()


NameError: name 'ds' is not defined

In [ ]:

ect = encoder.layer(test_batch,test_batch.batch).unsqueeze(1)


with torch.no_grad():
    recon_pts = encoder(ect)


# recon_ect_vae, theinput, z_mean, z_log_var = vae_litmodel.forward(ect)

# Undo the VAE transform
# recon_ect_vae = (recon_ect_vae + 1 ) / 2

# with torch.no_grad():
#     recon_vae_pts = ect_encoder_litmodel.model.forward(recon_ect_vae).cpu().detach().numpy()


pl = pv.Plotter(shape=(1, 4), window_size=[1600, 400],border=False,polygon_smoothing=True)


for idx in range(4):
    # points_vae = recon_vae_pts[idx].reshape(-1, 3)
    # pl.subplot(0, idx)
    # actor = pl.add_points(
    #     points_vae,
    #     style="points",
    #     emissive=False,
    #     show_scalar_bar=False,
    #     render_points_as_spheres=True,
    #     scalars=points_vae[:, 2],
    #     point_size=5,
    #     ambient=0.2, 
    #     diffuse=0.8, 
    #     specular=0.8,
    #     specular_power=40, 
    #     smooth_shading=True
    # )

    points = recon_pts[idx].reshape(-1, 3).cpu().detach().numpy()
    pl.subplot(0, idx)
    actor = pl.add_points(
        points,
        style="points",
        emissive=False,
        show_scalar_bar=False,
        render_points_as_spheres=True,
        scalars=points[:, 2],
        point_size=2,
        ambient=0.2, 
        diffuse=0.8, 
        specular=0.8,
        specular_power=40, 
        smooth_shading=True
    )



pl.background_color = "w"
pl.link_views()
pl.camera_position = "yz"
pos = pl.camera.position
pl.camera.position = (pos[0],pos[1],pos[2]+3)
pl.camera.azimuth = -45
pl.camera.elevation = 10

# create a top down light
light = pv.Light(position=(0, 0, 3), positional=True,
                cone_angle=50, exponent=20, intensity=.2)
pl.add_light(light)
pl.camera.zoom(1.3)
pl.screenshot("./figures/img/topological/reconstructed_pointcloud.png",transparent_background=True,scale=2)
pl.show()



Widget(value='<iframe src="http://localhost:50232/index.html?ui=P_0x2487f097cd0_0&reconnect=auto" class="pyvis…

# Interpolate between the ECT's 

In [ ]:
int_to_name = {
    0: "Sphere",
    1: "Torus",
    2: "Cube",
    3: "Mobius"
}

START = 1
END = 3
STEPS = 32
ect = encoder.layer(test_batch,test_batch.batch).unsqueeze(1)

with torch.no_grad():
    t = torch.tensor(np.linspace(0,1,STEPS,endpoint=True)).view(-1,1,1).cuda()
    ect_interp = (t * ect[START].repeat((STEPS,1,1)) + (1-t) * ect[END].repeat((STEPS,1,1))).to(torch.float32)
    ect_interp_pc = encoder(ect_interp.unsqueeze(1))


torch.save(ect_interp_pc.detach(),f"interpolation_manifold_{int_to_name[END].lower()}_to_{int_to_name[START].lower()}.pt")

# pl = pv.Plotter(shape=(1, 10), window_size=[1600, 400],border=False,polygon_smoothing=True)

# for idx in range(10):

#     points = ect_interp_pc[idx].reshape(-1, 3).cpu().detach().numpy()
#     pl.subplot(0, idx)
#     actor = pl.add_points(
#         points,
#         style="points",
#         emissive=False,
#         show_scalar_bar=False,
#         render_points_as_spheres=True,
#         scalars=points[:, 2],
#         point_size=6,
#         ambient=0.2, 
#         diffuse=0.8, 
#         specular=0.8,
#         specular_power=40, 
#         smooth_shading=True
#     )



# pl.background_color = "w"
# pl.link_views()
# pl.camera_position = "yz"
# pos = pl.camera.position
# pl.camera.position = (pos[0],pos[1],pos[2]+3)
# pl.camera.azimuth = -45
# pl.camera.elevation = 10

# # create a top down light
# light = pv.Light(position=(0, 0, 3), positional=True,
#                 cone_angle=50, exponent=20, intensity=.2)
# pl.add_light(light)
# pl.camera.zoom(1.3)
# pl.screenshot("./figures/img/topological/interpolate_mobius_torus_pointcloud.png",transparent_background=True,scale=2)
# pl.show()


## Sample from VAE and reconstruct points 

In [ ]:
samples = vae_litmodel.model.sample(64, "cuda:0")
samples = (samples + 1 ) / 2

n_images = 2

fig, axes = plt.subplots(
    nrows=1, ncols=n_images, sharex=True, sharey=True, figsize=(4,4)
)
fig.subplots_adjust(wspace=0.05,hspace=0.05)

for sample, ax in zip(samples,axes.T):
    ax.imshow(sample.cpu().detach().squeeze().numpy(),cmap="bone",vmin=-0.5,vmax=1.5)
    ax.axis("off")

plt.savefig("./figures/img/topological/generated_samples.svg",transparent=True)

NameError: name 'vae_litmodel' is not defined

In [ ]:

import pyvista as pv

pl = pv.Plotter(shape=(1, 2), window_size=[400, 200],border=False,polygon_smoothing=True)

with torch.no_grad():
    batch_decoded = ect_encoder_litmodel.model.forward(samples)

batch_decoded = batch_decoded.cpu().detach().numpy()

points = batch_decoded[0].reshape(-1, 3)
pl.subplot(0,0)
actor = pl.add_points(
    points,
    style="points",
    emissive=False,
    show_scalar_bar=False,
    render_points_as_spheres=True,
    scalars=points[:, 2],
    point_size=5,
    ambient=0.2, 
    diffuse=0.8, 
    specular=0.8,
    specular_power=40, 
    smooth_shading=True
)


points = batch_decoded[1].reshape(-1, 3)
pl.subplot(0,1)
actor = pl.add_points(
    points,
    style="points",
    emissive=False,
    show_scalar_bar=False,
    render_points_as_spheres=True,
    scalars=points[:, 2],
    point_size=5,
    ambient=0.2, 
    diffuse=0.8, 
    specular=0.8,
    specular_power=40, 
    smooth_shading=True
)



pl.background_color = "w"
pl.link_views()
pl.camera_position = "yz"
pos = pl.camera.position
pl.camera.position = (pos[0],pos[1],pos[2]+3)
pl.camera.azimuth = -45
pl.camera.elevation = 10

# create a top down light
light = pv.Light(position=(0, 0, 3), positional=True,
                cone_angle=50, exponent=20, intensity=.2)
pl.add_light(light)
pl.camera.zoom(1.3)
pl.screenshot("./figures/img/topological/generated_samples.png",transparent_background=True,scale=2)
pl.show()

Widget(value='<iframe src="http://localhost:64571/index.html?ui=P_0x22242b8fc40_3&reconnect=auto" class="pyvis…